In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pickle
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from IPython.display import SVG

from pydrake.examples.quadrotor import QuadrotorGeometry
from pydrake.geometry import MeshcatVisualizerCpp, Rgba, StartMeshcat
from pydrake.geometry.optimization import HPolyhedron, VPolytope
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.perception import PointCloud
from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem

from models.building_generation import *
from gcs.bezier import BezierGCS
from gcs.rounding import *

g_lic = GurobiSolver.AcquireLicense()
m_lic = MosekSolver.AcquireLicense()

In [ ]:
def generate_buildings(save_location, num_buildings):
    start = np.array([-1, -1])
    goal = np.array([2, 1])
    building_shape = (3, 3)
    
    for ii in range(num_buildings):
        file_location = save_location + "/room_" + str(ii).zfill(3)
        
        if not os.path.exists(file_location):
            os.makedirs(file_location)
        
        grid, outdoor_edges, wall_edges = generate_grid_world(shape=building_shape, start=start, goal=goal)
        regions = compile_sdf(file_location + "/building.sdf", grid, start, goal, outdoor_edges, wall_edges)
        with open(file_location + '/regions.reg', 'wb') as f:
            pickle.dump(regions, f)
            
def build_bezier_gcs(regions, solver):
    order = 7
    continuity = 4
    vel_limit = 10 * np.ones(3)
    hdot_min = 1e-3
    weights = {"time": 1., "norm": 1.}
    max_paths = 10 # default
    max_trials = 100 # default
    rounding_seed = 0
    
    gcs = BezierGCS(regions, order, continuity, hdot_min=hdot_min, full_dim_overlap=True)
    gcs.addTimeCost(weights["time"])
    gcs.addPathLengthCost(weights["norm"])
    gcs.addVelocityLimits(-vel_limit, vel_limit)
    gcs.setPaperSolverOptions()
    gcs.setSolver(solver)
    gcs.setRoundingStrategy(randomForwardPathSearch, max_paths=max_paths, max_trials=max_trials, seed=rounding_seed)
#     regularization = 1e-3
#     for derivative in [2, 3, 4]:
#         gcs.addDerivativeRegularization(regularization, regularization, derivative)

    return gcs
            
def plan_through_buildings(save_location, num_buildings, solve_relaxation=True, file_addendum=None, solver=None):
    start = np.array([-1, -1])
    goal = np.array([2, 1])
    start_pose = np.r_[(start-start)*5, 1.]
    goal_pose = np.r_[(goal-start)*5., 1.]
    
    if solver is None:
        solver = MosekSolver()
    
    for ii in range(num_buildings):
        file_location = save_location + "/room_" + str(ii).zfill(3)
        
        with open(file_location + "/regions.reg", "rb") as f:
            regions = pickle.load(f)
        
        start_setup = time.time()
        gcs = build_bezier_gcs(regions)
        setup_time = time.time() - start_setup
        
        planning_results = dict()
        planning_results["order"] = gcs.order
        planning_results["continuity"] = gcs.continuity
        planning_results["start_pose"] = start_pose
        planning_results["goal_pose"] = goal_pose
        planning_results["setup_time"] = setup_time
        
        start_time = time.time()
        output = gcs.SolvePath(start_pose, goal_pose, solve_relaxation, False,
                                     zero_deriv_boundary=3, preprocessing=True)
        b_traj, result, best_result, hard_results, statistics = output
        solve_time = time.time() - start_time
        
        print("Solve time for building", ii, ":", solve_time, flush=True)
        
        if solve_relaxation:
            planning_results["relaxation_time"] = solve_time
                
            planning_results["relaxation_solver_1_result"] = result.get_solution_result()
            planning_results["relaxation_solver_1_time"] = result.get_solver_details().optimizer_time
            planning_results["relaxation_solver_1_cost"] = result.get_optimal_cost()
            planning_results["relaxation_solver_1_solution"] = []
            for edge in gcs.gcs.Edges():
                edge_solution = {"name": edge.name(),
                                 "y_e": edge.GetSolutionPhiXu(result),
                                 "z_e": edge.GetSolutionPhiXv(result),
                                 "phi_e": result.GetSolution(edge.phi())}
                planning_results["relaxation_solver_1_solution"].append(edge_solution)

            if best_result is not None:
                planning_results["relaxation_solver_2_result"] = best_result.get_solution_result()
                planning_results["relaxation_solver_2_cost"] = best_result.get_optimal_cost()
                planning_results["relaxation_solver_2_solution"] = []
                for edge in gcs.gcs.Edges():
                    edge_solution = {"name": edge.name(),
                                     "y_e": edge.GetSolutionPhiXu(best_result),
                                     "z_e": edge.GetSolutionPhiXv(best_result),
                                     "phi_e": best_result.GetSolution(edge.phi())}
                    planning_results["relaxation_solver_2_solution"].append(edge_solution)

                max_hard_result_time = 0
                for result in hard_results:
                    if result.get_solver_details().optimizer_time > max_hard_result_time:
                        max_hard_result_time = result.get_solver_details().optimizer_time
                planning_results["relaxation_solver_2_time"] = max_hard_result_time
                planning_results["relaxation_solver_total_time"] = (planning_results["relaxation_solver_1_time"]
                                                                    + planning_results["relaxation_solver_2_time"])
            else:
                planning_results["relaxation_solver_2_result"] = None
                planning_results["relaxation_solver_2_time"] = np.nan
                planning_results["relaxation_solver_total_time"] = np.nan
                planning_results["relaxation_solver_2_cost"] = np.nan
                planning_results["relaxation_solver_2_solution"] = None

            traj_file = file_location + "/relaxation_traj.pkl"
            result_file = file_location + "/relaxation_plan_results.pkl"
            if file_addendum is not None:
                traj_file = file_location + "/relaxation_traj_" + file_addendum + ".pkl"
                result_file = file_location + "/relaxation_plan_results_" + file_addendum + ".pkl"
            with open(traj_file, "wb") as f:
                pickle.dump(b_traj, f, pickle.HIGHEST_PROTOCOL)
            with open(result_file, 'wb') as f:
                pickle.dump(planning_results, f)
                
        else:
            planning_results["mip_time"] = solve_time

            planning_results["mip_solver_result"] = result.get_solution_result()
            planning_results["mip_solver_time"] = result.get_solver_details().optimizer_time
            planning_results["mip_solver_cost"] = result.get_optimal_cost()
            planning_results["mip_solver_solution"] = []
            for edge in gcs.gcs.Edges():
                edge_solution = {"name": edge.name(),
                                 "y_e": edge.GetSolutionPhiXu(result),
                                 "z_e": edge.GetSolutionPhiXv(result),
                                 "phi_e": result.GetSolution(edge.phi())}
                planning_results["mip_solver_solution"].append(edge_solution)

            traj_file = file_location + "/mip_traj.pkl"
            result_file = file_location + "/mip_plan_results.pkl"
            if file_addendum is not None:
                traj_file = file_location + "/mip_traj_" + file_addendum + ".pkl"
                result_file = file_location + "/mip_plan_results_" + file_addendum + ".pkl"
            with open(traj_file, "wb") as f:
                pickle.dump(b_traj, f, pickle.HIGHEST_PROTOCOL)
            with open(result_file, 'wb') as f:
                pickle.dump(planning_results, f)

In [ ]:
runs = 100
save_location = "data/room_gen/final_stats"

np.random.seed(0)
generate_buildings(save_location, runs)

np.random.seed(0)
start_time = time.time()
plan_through_buildings(save_location, runs, solve_relaxation=True, solver=MosekSolver())
plan_through_buildings(save_location, runs, solve_relaxation=False, solver=MosekSolver())
# plan_through_buildings(save_location, runs, solve_relaxation=False, solver=GurobiSolver(), file_addendum="gurobi")
print("Solved", runs, "buildings in", np.round((time.time()-start_time)/60., 4), "minutes.")

In [ ]:
runs = 100
failed_solves = []

relax_costs = np.empty(runs - len(failed_solves))
rounded_costs = np.empty(runs - len(failed_solves))
mip_costs = np.empty(runs - len(failed_solves))
relaxation_solver_time = np.empty(runs - len(failed_solves))
relaxation_time = np.empty(runs - len(failed_solves))
mip_solver_time = np.empty(runs - len(failed_solves))
mip_time = np.empty(runs - len(failed_solves))

ii = 0
for index in range(runs):
    if index in failed_solves:
        continue
    save_location = "data/room_gen/final_stats/room_" + str(index).zfill(3)
    with open(save_location + '/relaxation_plan_results.pkl', "rb") as f:
        data = pickle.load(f)
        relax_costs[ii] = data["relaxation_solver_1_cost"]
        rounded_costs[ii] = data["relaxation_solver_2_cost"]
        relaxation_solver_time[ii] = data["relaxation_solver_total_time"]
        relaxation_time[ii] = data["relaxation_time"]
        
    with open(save_location + '/mip_plan_results.pkl', "rb") as f:
        data = pickle.load(f)
        mip_costs[ii] = data["mip_solver_cost"]
        mip_solver_time[ii] = data["mip_solver_time"]
        mip_time[ii] = data["mip_time"]
    with open(save_location + '/mip_plan_results_gurobi.pkl', "rb") as f:
        data = pickle.load(f)
        if not np.isnan(data["mip_solver_cost"]):
            if mip_costs[ii] > data["mip_solver_cost"]:
                mip_costs[ii] = data["mip_solver_cost"]
                mip_solver_time[ii] = data["mip_solver_time"]
                mip_time[ii] = data["mip_time"]
        
    ii += 1
    
mip_costs = np.minimum(mip_costs, rounded_costs)
    
rounding_gap = (rounded_costs-relax_costs)/relax_costs
relaxation_gap = (mip_costs - relax_costs)/mip_costs
solution_gap = (rounded_costs-mip_costs)/mip_costs

optimality_tolerance = 0.01

print("Tight Rounding:", np.sum(rounding_gap < optimality_tolerance)/ii)
print("Mean rounding gap:", np.mean(rounding_gap))
print("Max rounding gap:", np.argmax(rounding_gap), np.max(rounding_gap))
print()

print("Tight relaxation:", np.sum(relaxation_gap < optimality_tolerance)/ii)
print("Mean relaxation gap:", np.mean(relaxation_gap))
print("Max relaxation gap:", np.argmax(relaxation_gap), np.max(relaxation_gap))
print()

print("Solved to optimality:", np.sum(solution_gap < optimality_tolerance)/ii)
print("Mean solution gap:", np.mean(solution_gap))
print("Max solution gap:", np.argmax(solution_gap), np.max(solution_gap))
print("Min solution gap:", np.argmin(solution_gap), np.min(solution_gap))
print(solution_gap[solution_gap < -1e-4])
print(np.arange(runs)[solution_gap < -1e-4])

print()
print("Mean Rounding Solver Time:", np.mean(relaxation_solver_time))
print("Mean MIP Solver Time:", np.mean(mip_solver_time))
print("Mean MIP/Rounding Solver Time:", np.mean(mip_solver_time/relaxation_solver_time))


In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

meshcat.SetProperty("/Grid", "visible", False)
meshcat.SetProperty("/Axes", "visible", False)
meshcat.SetProperty("/Lights/AmbientLight/<object>", "intensity", 0.8)
meshcat.SetProperty("/Lights/PointLightNegativeX/<object>", "intensity", 0)
meshcat.SetProperty("/Lights/PointLightPositiveX/<object>", "intensity", 0)

In [ ]:
start=np.array([-1, -1])
goal=np.array([2, 1])
grid, indoor_edges, wall_edges = generate_grid_world(shape=(3,3), start=start, goal=goal, seed=42)
# draw_grid_world(grid, start, goal, indoor_edges, wall_edges)

regions = compile_sdf("models/room_gen/building.sdf", grid, start, goal, indoor_edges, wall_edges, seed=42)

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
model_id = parser.AddModelFromFile("models/room_gen/building.sdf")

plant.Finalize()

MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)

# if running_as_notebook:
simulator.set_target_realtime_rate(1.0)

# Set the initial conditions
context = simulator.get_mutable_context()

meshcat.Delete()
for ii in range(len(regions)):
    v = VPolytope(regions[ii])
    meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                            ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))

# Simulate
simulator.AdvanceTo(0.1)

In [ ]:
class FlatnessInverter(LeafSystem):
    def __init__(self, traj, animator, t_offset=0):
        LeafSystem.__init__(self)
        self.traj = traj
        self.port = self.DeclareVectorOutputPort("state", 12, self.DoCalcState, {self.time_ticket()})
        self.t_offset = t_offset
        self.animator = animator
        
    def DoCalcState(self, context, output):
        t = context.get_time() + self.t_offset - 1e-4
        
        q = np.squeeze(self.traj.value(t))
        q_dot = np.squeeze(self.traj.EvalDerivative(t))
        q_ddot = np.squeeze(self.traj.EvalDerivative(t, 2))
        
        fz = np.sqrt(q_ddot[0]**2 + q_ddot[1]**2 + (q_ddot[2] + 9.81)**2)
        r = np.arcsin(-q_ddot[1]/fz)
        p = np.arcsin(q_ddot[0]/fz)
        
        output.set_value(np.concatenate((q, [r, p, 0], q_dot, np.zeros(3))))
        
        if self.animator is not None:
            frame = animator.frame(context.get_time())
            animator.SetProperty(frame, "/Cameras/default/rotated/<object>", "position", [-2.5, 4, 2.5])
            animator.SetTransform(frame, "/drake", RigidTransform(-q))

In [ ]:
view_relaxation = False
view_regions = False
view_traces = True
track_uav = False
save_html = False
room = 12

save_location = "data/room_gen/seed_0_statistics_hdot/room_" + str(room).zfill(3)
rounded_traj_file = save_location + "/relaxation_traj.pkl"
mip_traj_file = save_location + "/mip_traj.pkl"

# Load data from disk
shutil.copy(save_location + "/building.sdf", "models/room_gen/building.sdf")

if view_regions:
    with open(save_location + "/regions.reg", "rb") as f:
        regions = pickle.load(f)

if view_relaxation:
    with open(rounded_traj_file, "rb") as f:
        b_traj = pickle.load(f)
else:
    with open(mip_traj_file, "rb") as f:
        b_traj = pickle.load(f)

# Build and run Diagram
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
model_id = parser.AddModelFromFile("models/room_gen/building.sdf")

plant.Finalize()

meshcat_cpp = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

if b_traj is not None:
    animator = meshcat_cpp.StartRecording()
    if not track_uav:
        animator = None
    traj_system = builder.AddSystem(FlatnessInverter(b_traj, animator))
    quad = QuadrotorGeometry.AddToBuilder(builder, traj_system.get_output_port(0), scene_graph)
diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

meshcat.Delete()

if view_traces:
    with open(rounded_traj_file, "rb") as f:
        rounded_traj = pickle.load(f)
    with open(mip_traj_file, "rb") as f:
        mip_traj = pickle.load(f)
        
    samples = 15000
    rounded_knots = rounded_traj.vector_values(np.linspace(rounded_traj.start_time(), rounded_traj.end_time(), samples))
    mip_knots = mip_traj.vector_values(np.linspace(mip_traj.start_time(), mip_traj.end_time(), samples))
    
#     line_width = 10.0
#     meshcat.SetLine("trace/rounded", rounded_knots, line_width=line_width, rgba=Rgba(0.0, 0.0, 1.0, 1.0))
#     meshcat.SetLine("trace/mip", mip_knots, line_width=line_width, rgba=Rgba(1.0, 0.749, 0.0, 1.0))
    
    radius = 0.1
    rounded_pointcloud = PointCloud(samples)
    mip_pointcloud = PointCloud(samples)
    rounded_pointcloud.mutable_xyzs()[:] = rounded_knots[:]
    mip_pointcloud.mutable_xyzs()[:] = mip_knots[:]
    meshcat.SetObject("trace/rounded", rounded_pointcloud, radius, rgba=Rgba(0.0, 0.0, 1.0, 1.0))
    meshcat.SetObject("trace/mip", mip_pointcloud, radius, rgba=Rgba(1.0, 0.749, 0.0, 1.0))

if view_regions:
    for ii in range(len(regions)):
        v = VPolytope(regions[ii])
        meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                                ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))
        
# Simulate
if b_traj is not None:
    end_time = b_traj.end_time()
    simulator.AdvanceTo(end_time+0.05)
    meshcat_cpp.PublishRecording()
    
    if save_html:
        with open ("data/room_gen/uav_trajectory.html", "w") as f:
           f.write(meshcat.StaticHtml())
else:
    simulator.AdvanceTo(0.1)